# Implement VGG-Net

In [ ]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms

def conv3x3(in_channels: int, out_channels: int):
    return nn.Conv2d(in_channels=in_channels, out_channels=out_channels,
                     kernel_size=3, padding=1)

def max_pool_2d():
    return nn.MaxPool2d(kernel_size=2, stride=2)
  
class VGGlayer(nn.Module):
  
  def __init__(self, in_channels: int, out_channels: int, activated=True, 
              max_pool = False):
        super(VGGlayer, self).__init__()
      
        layers = [
            conv3x3(in_channels, out_channels),
            nn.ReLU(True),
        ] 
        
        if max_pool:
          layers += [max_pool_2d()]
          
        self.layer = nn.Sequential(*layers)
  
  def forward(self, x):
      return self.layer(x)


class VGG16(nn.Module):

    def __init__(self, in_channels: int = 3, num_classes: int = 1000):
        super(VGG16, self).__init__()

        self.conv_features = nn.Sequential(
            VGGlayer(in_channels, 64),
            VGGlayer(64, 64, max_pool=True),
            VGGlayer(64, 128),
            VGGlayer(128, 128, max_pool=True),
            VGGlayer(128, 256),
            VGGlayer(256, 256),
            VGGlayer(256, 256, max_pool=True),
            VGGlayer(256, 512),
            VGGlayer(512, 512),
            VGGlayer(512, 512, max_pool=True),
            VGGlayer(512, 512),
            VGGlayer(512, 512),
            VGGlayer(512, 512, max_pool=True)
        )

        self.classifier = nn.Sequential(
            nn.Linear(512 * 7 * 7, 4096),
            nn.ReLU(),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(),
            nn.Dropout(),
            nn.Linear(4096, num_classes),
        )

    def forward(self, x):
        x = self.conv_features(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x

# Forward Pass Debug
If it can process random data, then you're mostly alright :D

In [ ]:
import numpy as np

net = VGG16()
X = torch.rand((8, 3, 224, 224))
num_params = sum([np.prod(p.shape) for p in net.parameters()])

print(f"Number of parameters : {num_params}")
print('-'*50)
print(net)

# Let's train on CIFAR-10

In [ ]:
import torchvision
import torchvision.transforms as transforms

mean = (0.4913997551666284, 0.48215855929893703, 0.4465309133731618)
std  = (0.24703225141799082, 0.24348516474564, 0.26158783926049628)

transform = transforms.Compose(
    [transforms.Resize((224,224)),
     transforms.ToTensor(),
     transforms.Normalize(mean, std)])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64,
                                          shuffle=True, num_workers=8)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=64,
                                         shuffle=False, num_workers=8)
classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

In [ ]:
import matplotlib.pyplot as plt

# functions to show an image
def imshow(img):
    img = img * np.array(std)[:,None,None] + np.array(mean)[:,None,None] # unnormalize
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()

# get some random training images
dataiter = iter(trainloader)
images, labels = dataiter.next()
images, labels = images[:4], labels[:4]

# show images
imshow(torchvision.utils.make_grid(images))
print(' '.join('%5s' % classes[labels[j]] for j in range(4)))

In [ ]:
assert torch.cuda.is_available(), "Notebook is not configured properly!"
device = 'cuda:0'

net = VGG16(num_classes=10).to(device)
crit = nn.CrossEntropyLoss()
opt = torch.optim.SGD(net.parameters(), lr=0.001, momentum=0.9)
epochs = 10

In [ ]:
%load_ext tensorboard
%tensorboard --logdir logs

In [ ]:
from torch.utils.tensorboard import SummaryWriter
from datetime import datetime
from tqdm.notebook import tqdm

now = datetime.now()
train_name = f'{now.hour}:{now.minute}:{now.second}/'
writer = SummaryWriter('./logs/' + train_name)

for e in range(epochs):
  for i, (x, y) in enumerate(tqdm(trainloader)):
    x, y = x.to(device), y.to(device)
    y_pred = net(x)
    loss = crit(y_pred, y)
    # write value on Tensorboard
    if i % 50 == 0:
      writer.add_scalar('Loss/train', loss.cpu().item(), i + e * len(trainloader))
    
    opt.zero_grad()
    loss.backward()
    opt.step()
  corr = 0
  for x, y in testloader:
    x, y = x.to(device), y.to(device)
    y_pred = net(x)
    corr += (torch.max(y_pred, 1)[1] == y).sum()
  print(f"Accuracy for epoch {e}:{corr / len(testset)}")